# Interactive Lotka–Volterra Predator–Prey Model  

This notebook showcases the classical Lotka–Volterra model, with
interactive sliders so one can connect parameter values to biological
behaviour. 

---

## 1 Model equations  

For prey $x(t)$ and predator $y(t)$ populations

$$
\begin{aligned}
\dot x &= \alpha x - \beta x y,\\[4pt]
\dot y &= \delta x y - \gamma y.
\end{aligned}
$$

Below are the definitions and units of the model parameters:

| Symbol | Meaning | Typical dimension |
|--------|---------|-------------------|
| $\alpha$ | prey intrinsic growth rate | $\text{time}^{-1}$ |
| $\beta$ | predation rate coefficient | $\text{time}^{-1}\,\text{predator}^{-1}$ |
| $\gamma$ | predator mortality rate | $\text{time}^{-1}$ |
| $\delta$ | predator reproduction coefficient | $\text{time}^{-1}\,\text{prey}^{-1}$ |

---

## 2 Mathematical analysis

Below are some useful mathematical results about the basic Lotka-Volterra model.

* Two equilibria (points where the rate of population change is zero) exist:

    1. Trivial extinction: $(0,0)$.  
    2. Coexistence equilibrium: $(x^*,y^*)=\Bigl(\frac{\gamma}{\delta},\;\frac{\alpha}{\beta}\Bigr).$

* To determine the stability of these equilibria, one can perform linear stability analysis (computing the eigenvalues of the Jacobian matrix around the equilibrium). Doing this about the coexistence equilibrium shows purely imaginary eigenvalues. The corresponding trajectories are **closed orbits** — perpetual cycles.

* This system has a conserved quantity that will be useful in examining cycling of predator and prey populations: $H(x,y)=\delta x - \gamma\ln x + \beta y - \alpha\ln y.$ $H(x,y)$ is determined by initial conditions, and will not change as the system evolves. This quantity is also known as a "first integral".

---

## 3 Biological interpretation  

* When prey are plentiful ($x>x^*$) predators grow; when predators
  overshoot ($y>y^*$) prey decline, producing **limit cycles**.
* The area enclosed by an orbit is set by initial conditions via $H$;
  parameters tilt or stretch the phase portrait.

---

## 4 Interactive exploration  

* **Left panel:** time‑series of $x(t)$ and $y(t)$.  
* **Right panel:** phase portrait of dynamics. Dashed lines are nullclines, lines where the rate of change of a population is zero. Value of conserved‑quantity $H$ is shown. Sliders change $\alpha,\beta,\gamma,\delta$ and
  starting values to illustrate how amplitude and period of the population cycles respond.



In [2]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import ipywidgets as wd

plt.rcParams["figure.dpi"] = 120

def lv_rhs(t, z, α, β, γ, δ):
    x, y = z
    return [α*x - β*x*y, δ*x*y - γ*y]

def simulate(
    α=1.0, β=0.1, γ=1.5, δ=0.075,
    x0=30.0, y0=4.0, t_end=60
):
    # integrate
    sol = solve_ivp(lv_rhs, (0, t_end), [x0, y0],
                    args=(α, β, γ, δ),
                    max_step=0.1, dense_output=True)
    t = np.linspace(0, t_end, 1000)
    x, y = sol.sol(t)

    # conserved H for trajectory
    H_traj = δ*x - γ*np.log(x) + β*y - α*np.log(y)
    H_val = H_traj[0]

    # plot
    fig, (ax_ts, ax_pp) = plt.subplots(1, 2, figsize=(11,4))

    # time‑series
    ax_ts.plot(t, x, label='prey $x$')
    ax_ts.plot(t, y, label='predator $y$')
    ax_ts.set_xlabel('time')
    ax_ts.set_ylabel('population')
    ax_ts.legend()
    ax_ts.set_title('Time‑series')

    # phase portrait
    # nullclines
    x_nc = np.linspace(0.1, max(x)*1.2, 400)
    ax_pp.plot(x_nc, α/β*np.ones_like(x_nc),
               'r--', lw=1, label='$\\dot y=0$')
    y_nc = np.linspace(0.1, max(y)*1.2, 400)
    ax_pp.plot(γ/δ*np.ones_like(y_nc), y_nc,
               'b--', lw=1, label='$\\dot x=0$')
    # trajectory
    ax_pp.plot(x, y, color='k')
    ax_pp.scatter([x0], [y0], color='red', zorder=5)
    ax_pp.set_xlabel('prey $x$')
    ax_pp.set_ylabel('predator $y$')
    ax_pp.set_xlim(0, max(x)*1.1)
    ax_pp.set_ylim(0, max(y)*1.1)
    ax_pp.set_title(f'Phase portrait (H={H_val:.2f})')
    ax_pp.legend(fontsize=7)

    plt.tight_layout()
    plt.show()

# Widgets
ui = wd.interactive(
    simulate,
    α = wd.FloatSlider(min=0.2, max=2.0, step=0.05, value=1.0,
                       description='α (prey growth)'),
    β = wd.FloatSlider(min=0.01, max=0.5, step=0.01, value=0.1,
                       description='β (pred rate)'),
    γ = wd.FloatSlider(min=0.2, max=3.0, step=0.05, value=1.5,
                       description='γ (pred mortality)'),
    δ = wd.FloatSlider(min=0.01, max=0.3, step=0.005, value=0.075,
                       description='δ (pred reproduction)'),
    x0 = wd.FloatSlider(min=5, max=60, step=1, value=30,
                       description='prey $x_0$'),
    y0 = wd.FloatSlider(min=1, max=60, step=1, value=4,
                       description='pred $y_0$')
)
display(ui)

interactive(children=(FloatSlider(value=1.0, description='α (prey growth)', max=2.0, min=0.2, step=0.05), Floa…

---

## 5 Why study the Lotka–Volterra cycles?  

* **Ecological baselines:** Real predator–prey systems rarely follow
  perfect cycles, but LV provides a *simple null model* to build upon.
  
* **Wide applicability:** The same equations appear in phage-bacteria,
  host–parasite, immune–pathogen, and even supply–demand dynamics. The
  conserved quantity explains persistent oscillations.

* **Pedagogic clarity:** Because LV is analytically tractable,
  students can see how geometry (nullclines & first integrals) links to
  biology (growth, predation, mortality). Interactive sliders make
  sensitivity to parameters tangible.

